# 데이터 전처리

In [1]:
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

import plotly.express as px
import plotly.graph_objs as go 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import numpy as np
%matplotlib inline
sns.set_style("darkgrid")

In [3]:
# 파이썬 시각화 패키지 불러오기
import matplotlib.pyplot as plt
%matplotlib inline


# 사용자 운영체제 확인
import platform
platform.system()

'Windows'

In [4]:
# 운영체제별 한글 폰트 설정
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')

plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정


# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

In [5]:
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [6]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm


font_path = r'C:/Windows/Fonts/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=18)

In [7]:
# 데이터 불러오기 
df1 = pd.read_csv("C:/Users/sweet/OneDrive/바탕 화면/국민건강보험공단_진료내역정보_20191231/T20_2019_1백만.1.csv",encoding='euc-kr')
df2 = pd.read_csv("C:/Users/sweet/OneDrive/바탕 화면/국민건강보험공단_진료내역정보_20191231/T20_2019_1백만.2.csv",encoding='euc-kr')
df3 = pd.read_csv("C:/Users/sweet/OneDrive/바탕 화면/국민건강보험공단_진료내역정보_20191231/T20_2019_1백만.3.csv",encoding='euc-kr')

In [8]:
# 데이터 합치기
df = pd.concat([df1, df2, df3])

In [9]:
# 입원여부열 추가
df['입원여부'] = np.where(df['서식코드'] == 2, 1, 2)

# 서식코드가 2이면 입원(1), 아니면(2)
# df['입원여부'] = df.서식코드.apply(lambda x: 1 if x == 2 else 2)

In [10]:
# 주상병코드에 연결하여 질병코드 추가
df['질병코드'] = df['주상병코드'].str[:1]

In [11]:
# 시도코드에 연결하여 시도명 추가
df_sido = pd.read_csv("sido.txt", sep = "|")
df_sido.columns = ['시도코드', '시도명']
df = pd.merge(df,df_sido, how='outer',on=['시도코드'])

In [12]:
# 연령대코드에 연결하여 연령대 추가
df_age = pd.read_csv("age.txt", sep = '\t')
df_age.columns = ['연령대', '연령대코드']
df = pd.merge(df,df_age, how='left',on=['연령대코드'])

In [13]:
# 진료과목코드에 연결하여 진료과 추가
df_dep = pd.read_csv("department.txt", sep = '\t')
df_dep.columns = ['진료과목코드', '진료과']
df = pd.merge(df,df_dep, how='left',on=['진료과목코드'])

In [14]:
df.tail(2)

,기준년도,가입자 일련번호,진료내역일련번호,성별코드,연령대코드,시도코드,요양개시일자,서식코드,진료과목코드,주상병코드,...,심결요양급여비용총액,심결본인부담금,심결보험자부담금,총처방일수,데이터 기준일자,입원여부,질병코드,시도명,연령대,진료과
13178343,2019,999978,101332700,1,9,43,20190102,3,1,K642,...,19690,5900,13790,5,20201222,2,K,충청북도,40~44세,내과
13178344,2019,999978,110275136,1,9,43,20190612,3,12,H1688,...,25660,7600,18060,3,20201222,2,H,충청북도,40~44세,안과


In [15]:
# 필요없는 열 삭제
df.drop(['기준년도', '데이터 기준일자', '요양개시일자', '심결보험자부담금'], axis = 1, inplace = True)

In [16]:
# 입원 자료만 데이터프레임 df_hos로
df_hos = df[df['입원여부'] == 1]
df_hos.head(3)

,가입자 일련번호,진료내역일련번호,성별코드,연령대코드,시도코드,서식코드,진료과목코드,주상병코드,부상병코드,요양일수,입내원일수,심결가산율,심결요양급여비용총액,심결본인부담금,총처방일수,입원여부,질병코드,시도명,연령대,진료과
3,1,109043837,2,12,46,2,1,E1164,N308,7,21,0.25,861590,205810,0,1,E,전라남도,55~59세,내과
38,1,105624357,2,12,46,2,5,M751,M750,12,15,0.25,2899570,638600,0,1,M,전라남도,55~59세,정형외과
63,93,110690589,2,16,46,2,1,N10,J0100,7,14,0.30,2178170,628230,0,1,N,전라남도,75~79세,내과


In [17]:
# 외래 자료만 데이터프레임 df_non_hos로
df_non_hos = df[df['입원여부'] == 2]
df_non_hos.head(3)

,가입자 일련번호,진료내역일련번호,성별코드,연령대코드,시도코드,서식코드,진료과목코드,주상병코드,부상병코드,요양일수,입내원일수,심결가산율,심결요양급여비용총액,심결본인부담금,총처방일수,입원여부,질병코드,시도명,연령대,진료과
0,1,104243474,2,12,46,3,5,M751,M750,1,1,0.25,19240,9600,0,2,M,전라남도,55~59세,정형외과
1,1,107479370,2,12,46,3,1,E1164,N308,1,1,0.25,145680,72800,0,2,E,전라남도,55~59세,내과
2,1,106881883,2,12,46,3,1,N308,E1164,1,1,0.25,13240,6600,3,2,N,전라남도,55~59세,내과


In [22]:
# 전체 요양 내역의 unique한 질병코드
jin = df['질병코드'].unique()
len(jin)

22

In [23]:
# 입원 내역의 unique한 질병코드 
jin_hos = df_hos['질병코드'].unique()
len(jin_hos)

22

In [24]:
# 진료과 unique
dep = df['진료과'].unique()
len(dep)

33

In [25]:
# 입원환자 진료과 unique
dep_hos = df_hos['진료과'].unique()
len(dep_hos)

32

In [26]:
# 데이터 타입 확인 
df_hos.dtypes

가입자 일련번호        int64
진료내역일련번호        int64
성별코드            int64
연령대코드           int64
시도코드            int64
서식코드            int64
진료과목코드          int64
주상병코드          object
부상병코드          object
요양일수            int64
입내원일수           int64
심결가산율         float64
심결요양급여비용총액      int64
심결본인부담금         int64
총처방일수           int64
입원여부            int32
질병코드           object
시도명            object
연령대            object
진료과            object
dtype: object

In [27]:
# object를 category로 변환
df['시도명'] = df['시도명'].astype('category')
df['연령대'] = df['연령대'].astype('category') 
df['진료과'] = df['진료과'].astype('category') 
df['질병코드'] = df['질병코드'].astype('category')  

In [28]:
# 데이터 타입 확인 
df_hos.dtypes

가입자 일련번호        int64
진료내역일련번호        int64
성별코드            int64
연령대코드           int64
시도코드            int64
서식코드            int64
진료과목코드          int64
주상병코드          object
부상병코드          object
요양일수            int64
입내원일수           int64
심결가산율         float64
심결요양급여비용총액      int64
심결본인부담금         int64
총처방일수           int64
입원여부            int32
질병코드           object
시도명            object
연령대            object
진료과            object
dtype: object

In [30]:
# 비율 계산을 위한 데이터프레임 - 지역별 입원/외래 건수
df_loc = pd.crosstab(index = df['시도명'], columns = df["입원여부"])
df_loc.reset_index(inplace = True)
df_loc.columns = ['시도', '1(입원)', '2(외래)']
df_loc.head(5)

,시도,1(입원),2(외래)
0,강원도,7928,356466
1,경기도,57946,3143574
2,경상남도,24848,839227
3,경상북도,17174,690849
4,광주시,11748,337543


In [31]:
# 비율 계산
df_loc['입원비율'] = df_loc['2(외래)'] / (df_loc['1(입원)'] + df_loc['2(외래)'])
df_loc['외래비율'] = df_loc['1(입원)'] / (df_loc['1(입원)'] + df_loc['2(외래)'])
df_loc.head(5)

,시도,1(입원),2(외래),입원비율,외래비율
0,강원도,7928,356466,0.978243,0.021757
1,경기도,57946,3143574,0.981900,0.018100
2,경상남도,24848,839227,0.971243,0.028757
3,경상북도,17174,690849,0.975744,0.024256
4,광주시,11748,337543,0.966366,0.033634


In [32]:
# 비율 계산을 위한 데이터프레임 - 진료과별 입원/외래 건수
df_dep = pd.crosstab(index = df['진료과'], columns = df["입원여부"])
df_dep.reset_index(inplace = True)
df_dep.columns = ['진료과', '1(입원)', '2(외래)']
df_dep.head(5)

,진료과,1(입원),2(외래)
0,가정의학과,18220,361533
1,결핵과,69,1017
2,구강악안면외과,9,1
3,내과,68497,4319462
4,마취통증의학과,1398,194939


In [33]:
# 비율 계산
df_dep['입원비율'] = df_dep['2(외래)'] / (df_dep['1(입원)'] + df_dep['2(외래)'])
df_dep['외래비율'] = df_dep['1(입원)'] / (df_dep['1(입원)'] + df_dep['2(외래)'])
df_dep.head(5)

,진료과,1(입원),2(외래),입원비율,외래비율
0,가정의학과,18220,361533,0.952021,0.047979
1,결핵과,69,1017,0.936464,0.063536
2,구강악안면외과,9,1,0.100000,0.900000
3,내과,68497,4319462,0.984390,0.015610
4,마취통증의학과,1398,194939,0.992880,0.007120
